# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
import pickle

In [2]:
!pip install transformers==4.19.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 50.1 MB/s 
     |████████████████████████████████| 596 kB 49.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Connect to GPU

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda")

In [4]:
!nvidia-smi

Wed Jun 15 11:46:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install model

In [5]:
model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained('AnyaSchen/rugpt3_esenin').to(DEVICE)

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/980 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

# Fine-tuning step-by-step

## Add special tokens

In [7]:
SPECIAL_TOKENS = {'bos_token' : "<bos>", "eos_token" :"<eos>", 'pad_token':'<pad>'}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1024)

## Get data

In [8]:
!wget https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0 -O dataset.pkl

--2022-06-14 09:51:39--  https://www.dropbox.com/s/b6q4tokw0skjsfz/dataset.pkl?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/b6q4tokw0skjsfz/dataset.pkl [following]
--2022-06-14 09:51:39--  https://www.dropbox.com/s/raw/b6q4tokw0skjsfz/dataset.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc77f53c97a156a519d226f4dd0d.dl.dropboxusercontent.com/cd/0/inline/BnJU7cmP-PlzjldoTOhEjQbBAaEYsi5F77D2J1ofZZbEl81KzBghqi1b91HV67ToFCKlc7-OOhT-ntu2i-P21AsPWRbHVjuREUR5-JVcklUrYCA1hKKAxskPTgD_uuxg5cP-pUt-LP1QhfBaSrH3ahSo492bTM3ue_3l4rEOu2EoXA/file# [following]
--2022-06-14 09:51:40--  https://uc77f53c97a156a519d226f4dd0d.dl.dropboxusercontent.com/cd/0/inline/BnJU7cmP-PlzjldoTOhEjQbBAaEYsi5F77D2J1ofZZbEl81KzBghqi1b91HV67ToFC

In [9]:
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

## Greate a Dataset

In [10]:
import torch
torch.manual_seed(42)
from torch.utils.data import Dataset # this is the pytorch class import

class myDataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=150):

    self.tokenizer = tokenizer # the gpt2 tokenizer we instantiated
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:
      """
      This loop will iterate through each entry in the flavour text corpus.
      For each bit of text it will prepend it with the start of text token,
      then append the end of text token and pad to the maximum length with the 
      pad token. 
      """

      encodings_dict = tokenizer('<bos>'+ txt + '<eos>', 
                                 truncation=True, 
                                 max_length=max_length, 
                                 padding="max_length")
      
      """
      Each iteration then appends either the encoded tensor to a list,
      or the attention mask for that encoding to a list. The attention mask is
      a binary list of 1's or 0's which determine whether the langauge model
      should take that token into consideration or not. 
      """
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return {
        'input_ids': self.input_ids[idx],
        'attn_masks': self.attn_masks[idx]
    }

In [11]:
train_dataset = myDataset(dataset['esenin'], tokenizer)

## Add Datacollator

In [12]:
from transformers import TextDataset, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [13]:
train_dataset[10]

{'attn_masks': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([50257,  3096, 23678,  2300, 29541,  5740,   510, 30040,  3881,  5539,
          1244,   465,    16,   203,  2176,  1017,  2557,   329,  3881,   307,
         36805,   360,  9450,   346,  2067,   285,  3795,    18,   203,   618,
          7213,  7741,  9364,  2208,  9338,  1300,   508,   326,   330,   309,
           327,   944,    16,   203, 43332,   669,   906,  9226, 43512,    16,
           901,   466,   322,   639,   282, 12825,    18, 50258

## Training

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./drive/MyDrive/esenin_checkouts_2/", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=15, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=150,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=2, # to make "virtual" batch size larger
    save_steps = 1000
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
trainer.train()

***** Running training *****
  Num examples = 1209
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 2265
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: attn_masks. If attn_masks are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Step,Training Loss
500,0.763000
1000,0.669700
1500,0.641300
2000,0.613000


Saving model checkpoint to ./drive/MyDrive/esenin_checkouts_2/checkpoint-1000
Configuration saved in ./drive/MyDrive/esenin_checkouts_2/checkpoint-1000/config.json
Model weights saved in ./drive/MyDrive/esenin_checkouts_2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./drive/MyDrive/esenin_checkouts_2/checkpoint-2000
Configuration saved in ./drive/MyDrive/esenin_checkouts_2/checkpoint-2000/config.json
Model weights saved in ./drive/MyDrive/esenin_checkouts_2/checkpoint-2000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2265, training_loss=0.6616058282241652, metrics={'train_runtime': 2383.8539, 'train_samples_per_second': 7.607, 'train_steps_per_second': 0.95, 'total_flos': 4933903826534400.0, 'train_loss': 0.6616058282241652, 'epoch': 15.0})

## Save models

In [25]:
trainer.save_model('/content/drive/MyDrive/esenin_model_2/')

Saving model checkpoint to /content/drive/MyDrive/esenin_model_2/
Configuration saved in /content/drive/MyDrive/esenin_model_2/config.json
Model weights saved in /content/drive/MyDrive/esenin_model_2/pytorch_model.bin


# Generation

In [8]:
# Пример вероятностного сэмплирвоания с ограничением
text = "И ветер дул, и небо моросило"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=3,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=100,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



И ветер дул, и небо моросило*,
Но никто из нас не думал о смерти.
Я помню праздник, праздник мая,
Наше пионерское братство.
Нам читали стихи об отваге,
И каждый горстью синей своей
Мял в мальчишеских руках гвозди.
Нам хотелось быть такими,
Как они… Да, такими мы были.
И потому, что мы были другие,
Пролетая над страной в пижаме,
